# Feature Engineering

Importing libraries

In [12]:
# feature_engineering.ipynb
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.abspath('..'))



In [13]:
from src.data_preprocessing import (
    build_feature_engineering_pipeline,
    save_pipeline
)

In [14]:

raw_path = "/Users/elbethelzewdie/Downloads/credit_risk_model/credit-risk-model/data/raw/data.csv"    # <-- change if needed

df = pd.read_csv(raw_path)
print("✅ Raw data loaded:", df.shape)

✅ Raw data loaded: (95662, 16)


In [15]:
print("📊 Data Overview")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

📊 Data Overview
Shape: (95662, 16)
Columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult']


In [16]:
# -------------------------------------------------------------------
# DEFINE YOUR COLUMNS (ADJUST THESE BASED ON YOUR DATA)
# -------------------------------------------------------------------
# Example - adjust based on your actual column names
CUSTOMER_ID_COL = 'CustomerId'
AMOUNT_COL = 'Amount'  # Your transaction amount column
DATE_COL = 'TransactionStartTime'


In [17]:
# Identify numeric and categorical columns from your data
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()


In [18]:
# Remove ID and date columns from feature lists
id_cols = [CUSTOMER_ID_COL, 'TransactionId', 'AccountId', 'SubscriptionId', 'BatchId', 'CountryCode']
date_cols = [DATE_COL]

In [19]:
# Clean feature lists
numeric_features = [col for col in numeric_cols 
                    if col not in id_cols + date_cols and col != AMOUNT_COL]
categorical_features = [col for col in categorical_cols 
                        if col not in id_cols + date_cols]

print(f"\n🔧 Column Configuration")
print(f"Customer ID column: {CUSTOMER_ID_COL}")
print(f"Amount column: {AMOUNT_COL}")
print(f"Date column: {DATE_COL}")
print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")


🔧 Column Configuration
Customer ID column: CustomerId
Amount column: Amount
Date column: TransactionStartTime
Numeric features: 3
Categorical features: 5


In [20]:
# -------------------------------------------------------------------
# BUILD AND RUN PIPELINE
# -------------------------------------------------------------------
print("\n🚀 Building Feature Engineering Pipeline")
print("=" * 50)

pipeline = build_feature_engineering_pipeline(
    customer_id_col=CUSTOMER_ID_COL,
    amount_col=AMOUNT_COL,
    date_col=DATE_COL,
    categorical_cols=categorical_features,
    numeric_cols=numeric_features,
    use_woe=False,
    woe_target_col=None,
    remove_outliers=True,
    scaling_method='standard'
)

print("Fitting and transforming data...")
X_features = pipeline.fit_transform(df)

print(f"\n✅ Feature Engineering Complete!")
print(f"   Original shape: {df.shape}")
print(f"   Engineered features shape: {X_features.shape}")
print(f"   Output type: {type(X_features)}")



🚀 Building Feature Engineering Pipeline
Fitting and transforming data...
  Outlier removal: 1829 rows removed (48.9%)

✅ Feature Engineering Complete!
   Original shape: (95662, 16)
   Engineered features shape: (1913, 55)
   Output type: <class 'pandas.core.frame.DataFrame'>


In [21]:
# -------------------------------------------------------------------
# CHECK THE OUTPUT
# -------------------------------------------------------------------
print("\n🔍 Checking Output")
print("=" * 50)

if isinstance(X_features, pd.DataFrame):
    print("✅ Output is a DataFrame")
    print(f"\n📋 First 5 rows:")
    print(X_features.head())
    
    print(f"\n📊 Column names (first 10):")
    for i, col in enumerate(X_features.columns[:10]):
        print(f"{i+1:2d}. {col}")
else:
    print("⚠️ Output is not a DataFrame")
    print(f"Type: {type(X_features)}")
    print(f"Shape: {X_features.shape}")


🔍 Checking Output
✅ Output is a DataFrame

📋 First 5 rows:
   num__Amount_sum  num__Amount_mean  num__Amount_count  num__Amount_std  \
2        -0.243171          0.889794          -0.587556         0.996790   
3        -0.737024         -1.150703           0.055768        -1.054675   
4        -0.243171          0.513586          -0.480336         0.816049   
6         0.658443          1.692369          -0.158673         0.800045   
7        -0.837986         -0.803139          -1.016439        -1.246365   

   num__Amount_min  num__Amount_max  num__Amount_median  num__Value_mean  \
2        -1.100234         0.356565            0.919637         0.980784   
3         0.805641        -0.980236           -0.968040        -1.145650   
4        -1.100234         0.356565            0.124826         0.699565   
6        -1.100234         0.356565            2.653771         1.251960   
7         1.221645        -1.058871           -0.597730        -0.932510   

   num__Value_std  num__Va

In [22]:
# -------------------------------------------------------------------
# SAVE RESULTS
# -------------------------------------------------------------------
print("\n💾 Saving Results")
print("=" * 50)

output_dir = "/Users/elbethelzewdie/Downloads/credit_risk_model/credit-risk-model/data/processed/"
os.makedirs(output_dir, exist_ok=True)

# Save engineered features
features_path = f"{output_dir}preprocessed.csv"

if isinstance(X_features, pd.DataFrame):
    X_features.to_csv(features_path, index=False)
else:
    pd.DataFrame(X_features).to_csv(features_path, index=False)

print(f"✅ Engineered features saved to: {features_path}")

# Save pipeline (NOW IT WILL WORK - no nested classes)
pipeline_path = f"{output_dir}feature_pipeline.pkl"
save_pipeline(pipeline, pipeline_path)

# Verify the saved file
print(f"\n📁 Verifying saved file:")
saved_df = pd.read_csv(features_path)
print(f"   Loaded shape: {saved_df.shape}")
print(f"   First 3 columns: {saved_df.columns.tolist()[:3]}")
print(f"   Sample values from first row:")
for col in saved_df.columns[:3]:
    print(f"     {col}: {saved_df[col].iloc[0]}")

print("\n🎉 Feature Engineering Pipeline Complete!")
print("=" * 50)


💾 Saving Results
✅ Engineered features saved to: /Users/elbethelzewdie/Downloads/credit_risk_model/credit-risk-model/data/processed/preprocessed.csv
✅ Pipeline saved to: /Users/elbethelzewdie/Downloads/credit_risk_model/credit-risk-model/data/processed/feature_pipeline.pkl

📁 Verifying saved file:
   Loaded shape: (1913, 55)
   First 3 columns: ['num__Amount_sum', 'num__Amount_mean', 'num__Amount_count']
   Sample values from first row:
     num__Amount_sum: -0.2431713870633395
     num__Amount_mean: 0.8897935064967836
     num__Amount_count: -0.5875563371533058

🎉 Feature Engineering Pipeline Complete!


- After feature engineering and preprocessing pipeline, the dataset now has 55 features.
- Originally, the raw dataset had 16 columns, so the pipeline has created many new features (aggregations, date features, outlier removal, etc.) and also applied One-Hot Encoding for categorical columns.

✅ Overall interpretation

- The feature engineering pipeline worked as intended: it expanded the dataset from 16 to 55 numeric features, removed outliers, and created meaningful features for modeling.

- High-variance features like transaction counts, sums, and coefficients of variation are likely strong predictors of credit risk.

- The absence of categorical features suggests all remaining columns are already numeric, ready for ML models like logistic regression, XGBoost, or neural networks.